# OPENAI API Load

### 1. 환경 설정 및 API 키 로드
이 셀은 필요한 라이브러리(`openai`, `python-dotenv`)를 설치하고 불러옵니다.
또한 `.env` 파일에서 `OPENAI_API_KEY`를 안전하게 로드하여 `openai` 라이브러리에 설정합니다.

In [30]:
import os
from dotenv import load_dotenv

# .env 파일에 정의된 환경 변수를 로드
load_dotenv()

# 환경 변수에서 OPENAI_API_KEY 값을 가져옴
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [31]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable


### 2. OpenAI API 호출
이 셀은 설정된 클라이언트를 사용하여 `gpt-4o-mini` 모델에게 "Hello!"라는 메시지를 보내고, 그 응답을 출력합니다.

In [32]:
from openai import OpenAI

# OpenAI 클라이언트 생성 (API 키는 환경 변수에서 자동으로 읽힘)
client = OpenAI()

# 첫 번째 대화 요청
response = client.chat.completions.create(
    model="gpt-4.1-mini",  # 사용할 모델 지정
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},  # 시스템 프롬프트
        {"role": "user", "content": "Hello! I'm John."}                 # 사용자 메시지
    ]
)

# response 객체 전체 출력 (Python 객체 형태)
print(response)

ChatCompletion(id='chatcmpl-CxoleQPcmQmt75bsynlK7wkd18Wz4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello John! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1768372538, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_376a7ccef1', usage=CompletionUsage(completion_tokens=10, prompt_tokens=22, total_tokens=32, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [33]:
# response를 JSON 형태로 보기 좋게 출력
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-CxoleQPcmQmt75bsynlK7wkd18Wz4",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello John! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1768372538,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_376a7ccef1",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 22,
    "total_tokens": 32,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [34]:
# 이전 대화 내용을 포함한 두 번째 요청
response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."},
        {"role": "assistant", "content": "Hello John! How can I assist you today?"},
        {"role": "user", "content": "Do you know my name?"}  # 모델이 문맥을 기억하는지 확인
    ]
)

# 두 번째 응답을 JSON 형태로 출력
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-CxoleUaIzh141z3vPfKRgEW3hwGyw",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Yes, you mentioned that your name is John. How can I help you, John?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1768372538,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_376a7ccef1",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 46,
    "total_tokens": 64,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [35]:
# ---------------------------
# 1️⃣ 스트리밍 방식 응답 받기
# ---------------------------

response = client.chat.completions.create(
    model="gpt-4.1-mini",  # 사용할 GPT 모델
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."}
    ],
    stream=True  # 응답을 한 번에 받지 않고, 조금씩 받기
)

# 스트리밍으로 오는 응답은 반복문으로 하나씩 처리
for chunk in response:
    choice = chunk.choices[0]  # 첫 번째 응답 조각 선택

    # finish_reason이 None이면 아직 말하는 중이라는 뜻
    if choice.finish_reason is None:
        # delta.content에 새로 생성된 텍스트가 들어 있음
        print(choice.delta.content, end="")  # 줄바꿈 없이 출력

Hello John! How can I assist you today?

In [36]:
# ---------------------------
# 2️⃣ 일반 방식 (한 번에 응답 받기)
# ---------------------------

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "user", "content": "Hello! I'm John."}
    ]
)

# 전체 응답을 JSON 형태로 보기 좋게 출력
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-CxolgVcz9eUDINNPTixZ0qHScyOVZ",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello John! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1768372540,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_376a7ccef1",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 12,
    "total_tokens": 22,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [37]:
# ---------------------------
# 3️⃣ 대화 문맥을 포함한 요청
# ---------------------------

# 이전 대화를 리스트로 직접 관리
messages = [
    {"role": "user", "content": "Hello! I'm John."},
    {"role": "assistant", "content": "Nice to meet you, John!"},
    {"role": "user", "content": "Do you know my name?"}
]

# 위 대화 내용을 모두 포함해서 다시 요청
response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages
)

# AI가 대화를 기억하는 것처럼 보이는 응답 확인
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-CxolgW5XGwD4qio3a9MgxJOc0l6MM",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Yes, you mentioned that your name is John. How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1768372540,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_376a7ccef1",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 33,
    "total_tokens": 50,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
